gemaakt voor LIWO door [David Haasnoot](d.haasnoot@hkv.nl)

In [1]:
import dotenv
from pathlib import Path

In [ ]:
from LDO_API.update_local_LDO_custom import haal_scenarios_op, haal_token_op, get_layer_names_from_scenario, export_uit_LDO_custom

Zie `readme.md` voor uitgebreid stappen plan voor het aanmaken van een api key.

In het kort
- Login & ga naar https://www.ldo.overstromingsinformatie.nl/auth/
- Onder `V1`, `POST auth/v1/personalapikeys`, klik op `try it out` en vervang body met:
```json
{
  "scope": "*:readwrite",
  "name": "personalAPIkeyLIWOexport...",
  "expiry_date": "2029-12-31T23:59:59.037Z",
  "revoked": false
}
```
- Pas de datum en name aan waar nodig, klik op `Excecute` in het blauw. In de response body staat: 
```json
{
...
  "key": "xxxxxx.yyyyyyyyyyyyyyy",
...
}
```
- Bewaar die hele `'key'` in een bestand die `.env` heet, zie `example.env` voor het formaat.

In [3]:
# haal de API key op uit de .env file
if dotenv.load_dotenv():
    environmental_variables = dotenv.dotenv_values()
    LDO_api_key = environmental_variables["LDO_api_key"]
    TENANT = int(environmental_variables.get("TENANT", 1))
# Of zet hier de API key handmatig in
# LDO_api_key ="abcd"
# TENANT : int  = 2 # 0, 1, 2 ...
headers = haal_token_op(LDO_api_key, tenant=TENANT)
current_dir = Path.cwd()

In [4]:
maximum = 10 #10_000
beschikbare_scenario_ids = haal_scenarios_op(maximum, headers)

In [5]:
len(beschikbare_scenario_ids)

100

Kies welke scenarios gedownload worden:

In [6]:
nieuwe_scenarios = beschikbare_scenario_ids[-2:]  # de laatste twee scenario's


Haal de bestands namen op

In [7]:
df_layer_names = get_layer_names_from_scenario( nieuwe_scenarios, headers=headers,)

In [8]:
df_layer_names

,0,1,2,3,4,5,6,7,8,9,10,11
3753,Mortality.tif,Total_damage.tif,Total_victims.tif,Total_affected.tif,gridmaxwaterdepth.zip,gridmaxflowvelocity.zip,scenario_3753_max_velocity.tiff,scenario_3753_max_waterdepth.tiff,None,None,None,None
3755,Mortality.tif,fls_import.zip,Total_damage.tif,Total_victims.tif,Total_affected.tif,gridmaxwaterdepth.zip,gridmaxflowvelocity.zip,scenario_3755_rise_period.tiff,scenario_3755_max_velocity.tiff,scenario_3755_rate_of_rise.tiff,scenario_3755_arrival_times.tiff,scenario_3755_max_waterdepth.tiff


Exporteert de gewenste bestanden (kan lang duren)

In [ ]:
lst_zips_nieuwe_export = export_uit_LDO_custom(
                                        df_layer_names=df_layer_names,
                                        work_dir=current_dir,
                                        headers=headers,
)

1it [00:02,  2.35s/it]